# processor: PrintResult

[PrintResult](../api/processors.rst#nornir_utils.plugins.processors.PrintResult) is a [processor](https://nornir.readthedocs.io/en/latest/tutorials/intro/processors.html) that prints results similarly to the `print_result` function, however, because this is a processor it is done asynchronously. For instance:

In [1]:
# NBVAL_IGNORE_OUTPUT

from nornir import InitNornir
from nornir_utils.plugins.processors import PrintResult

def dummy_task(task):
    return "hi!!!"

nr = InitNornir(
    inventory={
        "plugin":"YAMLInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        }
    }
)

nrp = nr.with_processors([PrintResult()])
_ = nrp.run(task=dummy_task)  # `_ = something` discards the result explicitly, not needed

**** dummy_task - Starting *****************************************************
vvvv dummy_task - dev1.group_1 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv dummy_task - dev2.group_1 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv dummy_task - dev3.group_2 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv dummy_task - dev4.group_2 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv dummy_task - dev5.no_group ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
**** dummy_task - Completed **************************

Notice we never printed the result explicitly, instead, the processors printed it on screen as soon as the results where made available. This is particularly useful if some hosts take long to complete while others are faster and you want feedback as soon as possible.

## Severity

By default `PrintResult` will only print `INFO` or above levels so the following code will skip intermediate results:

In [2]:
# NBVAL_IGNORE_OUTPUT
import logging

def complex_task(task):
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    task.run(task=dummy_task, severity_level=logging.DEBUG)
    return "I did a lot of things!!!"

nrp = nr.with_processors([PrintResult()])
_ = nrp.run(task=complex_task)  # `_ = something` discards the result explicitly, not needed

**** complex_task - Starting ***************************************************
vvvv complex_task - dev1.group_1 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv complex_task - dev3.group_2 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv complex_task - dev5.no_group ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv complex_task - dev4.group_2 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv complex_task - dev2.group_1 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
I did a lot of things!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

You could ask the processor to include those like this:

In [3]:
# NBVAL_IGNORE_OUTPUT
import logging

nrp = nr.with_processors([PrintResult(severity_level=logging.DEBUG)])
_ = nrp.run(task=complex_task)  # `_ = something` discards the result explicitly, not needed

**** complex_task - Starting ***************************************************
vvvv dummy_task - dev1.group_1 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv DEBUG
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv dummy_task - dev2.group_1 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv DEBUG
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv dummy_task - dev2.group_1 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv DEBUG
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv dummy_task - dev4.group_2 ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv DEBUG
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv dummy_task - dev5.no_group ** changed: False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv DEBUG
hi!!!
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
vvvv dummy_task - dev1.group_1 ** changed: False 

Note that there is no clear order, this is because the processor processes results as available.